In [1]:
import json
import sys
import os
import numpy as np
import skimage
from scipy import ndimage
from scipy import special
from scipy import sparse
import anndata
import nibabel as nib
import pandas as pd
import datashader as ds
import time
from sklearn.decomposition import PCA
import collections
import pickle as pkl
import zarr
import h5py
import anndata as ad

import ants
import nibabel as nib

from dredFISH.Analysis.__init__plots import *
sns.set_style('white')

import importlib
from dredFISH.Analysis import powerplots
importlib.reload(powerplots)
from dredFISH.Analysis import imageu
importlib.reload(imageu)
from dredFISH.Analysis import regu
importlib.reload(regu)
from dredFISH.Analysis import basicu
importlib.reload(basicu)

<module 'dredFISH.Analysis.basicu' from '/bigstore/GeneralStorage/fangming/projects/dredfish/packages/dredFISH/dredFISH/Analysis/basicu.py'>

In [3]:
ddir = '/bigstore/binfo/mouse/Brain/Sequencing/Allen_10X_CtxHippo/source/'
# ddir = '/bigstore/binfo/mouse/Brain/Sequencing/Allen_SmartSeq_CtxHippo/source/'
outdir = '/scratch/10x/'

# preload data

In [4]:
ti = time.time()
f = ddir + 'expression_matrix.hdf5'
with h5py.File(f, 'r') as fh:
    print(fh.keys())
    print(fh['data'].keys())
    
    print(fh['data']['counts'])
    print(fh['data']['gene'])# t is just the transpose
    print(fh['data']['samples'])
    print(fh['data']['shape'])
    
    print(fh['data']['shape'][()])
    
    # mat = fh['data']['counts'][:,:100] # [gene, cell]
    
    genes = fh['data']['gene'][()]
    genes = np.array([gene.decode() for gene in genes])
    
    cells = fh['data']['samples'][()]
    cells = np.array([cell.decode() for cell in cells])
    
genes_10x = genes.copy()

<KeysViewHDF5 ['data']>
<KeysViewHDF5 ['counts', 'gene', 'samples', 'shape']>
<HDF5 dataset "counts": shape (31053, 1169320), type "<i4">
<HDF5 dataset "gene": shape (31053,), type "|S30">
<HDF5 dataset "samples": shape (1169320,), type "|S36">
<HDF5 dataset "shape": shape (2,), type "<i4">
[1169320   31053]


In [5]:
# DPNMF matrix 
f = f'/bigstore/GeneralStorage/fangming/projects/dredfish/data/dpnmf.csv' 
pmat = pd.read_csv(f, index_col=0)
print(pmat.shape)

# for each allen cluster, get their DPNMF measurements
# remove genes with zero counts
pmat = pmat[pmat.sum(axis=1) > 0]
# 5818 genes, 5818 (all) overlap with 10x, 5576 overlap with Allen SS  
genes = np.intersect1d(pmat.index.values, genes_10x)
genes_idx = basicu.get_index_from_array(genes_10x, genes)
print(pmat.shape, genes_10x.shape, genes.shape)

pmat = pmat.loc[genes]
pmat

(9711, 24)
(5818, 24) (31053,) (5818,)


,RS0109_cy5,RS0175_cy5,RS0237_cy5,RS0307_cy5,RS0332_cy5,RS0384_atto565,RS0406_atto565,RS0451_atto565,RS0468_atto565,RS0548_atto565,...,RS643.0_alexa488,RS740.0_alexa488,RS810.0_alexa488,RSN9927.0_cy5,RSN2336.0_cy5,RSN1807.0_cy5,RSN4287.0_atto565,RSN1252.0_atto565,RSN9535.0_atto565,RS0095_cy5
0610010F05Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
0610012G03Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610043K17Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1110002L01Rik,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1110004E09Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zscan18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,7,0,0,0
Zswim1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Zswim8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4,0,0,0
Zyg11b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,27


# load, norm, project, save

In [6]:
%%time

ncells = len(cells)
# chunksize = 100
chunksize = 100000
curr_head = 0
print(ncells)
f = ddir + 'expression_matrix.hdf5'


with h5py.File(f, 'r') as fh:
    while curr_head < ncells:
        l, r = curr_head, curr_head+chunksize
        r = min(ncells, r)
        curr_head += chunksize
        print(l,r)
        
        # get matrix
        mat = fh['data']['counts'][:,l:r] # [gene, cell]
        
        # process
        Xcell = mat.T

        # # correct for library size (normalization) and projection 
        libsizes = Xcell.sum(axis=1)
        medsize = np.median(libsizes)

        Xcr_norm = (Xcell[:,genes_idx]/libsizes.reshape(-1,1))*medsize
        Xcr_norm = Xcr_norm.dot(pmat)

        # save to scratch
        np.save(f'{outdir}{l}.npy', Xcr_norm)
        
        # break


1169320
0 100000
100000 200000
200000 300000
300000 400000
400000 500000
500000 600000
600000 700000
700000 800000
800000 900000
900000 1000000
1000000 1100000
1100000 1169320
CPU times: user 14min 56s, sys: 20min 20s, total: 35min 17s
Wall time: 27min 22s
